# LIVRIA : Filtrage colllaboratif - Partie II

   Ce jupyter notebook contient le code permettant la prédiction des livres susceptibles de plaire à l'utilisateur. Nous entraînons et mesurons les prédictions ici, mais vous retrouverez la mise en forme des résultats de la prédiction dans le notebook Livra_recommender_system.ipynb.

   Nous avons dans un premier temps utilisé notre base de données issue du questionnaire afin de mettre en place des modèles de prédiction de thèmes, modèles basés sur des techniques de filtrage collaboratif. On a pu mesurer la perfomance des modèles et les comparer entre eux en se basant sur un type de mesure d'erreur entre les valeurs des données du set de test et les prédicions des différents modèles.
   Maintenant, nous allons pouvoir utiliser la base de données Goodbooks-10k pour réaliser le même travail de filtrage collaboratif mais cet fois-ci la prédiction sera portée sur des livres. 
   
* Le set de données de Goodbooks-10 :
http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/

Je précise que dans la dernière partie du notebook dataVizualisation&Cleaning_GoodBooks10k.ipynb, avons nettoyé le set de donnée "**ratings.csv**" que nous avons enregistré sous le nom "**df_notes.csv**" (cf. dossier ./data). Ainsi, nous avons gardé les livres les plus notés et les utilisateurs les plus actifs - ceux qui ont attribués le plus de notes - afin de faciliter le filtrage collaboratif et de contourner le problème de la taille du set complet qui était trop gros pour être utilisé entièrement ici.

## Import des librairies

On commence par importer les librairies utilisées dans ce notebook.

In [38]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

## Import des données

Vous pouvez retrouver l'ensemble des données utilisées dans le dossier './data'


In [39]:
# Lecture du fichier 'df_notes.csv'
df_notes = pd.read_csv('data/df_notes.csv', sep='\t')
# On supprime la colonne inutile :
del df_notes['Unnamed: 0']

In [40]:
### On montre un extrait des notes attribuées
df_notes.head(10)

,index_livre,book_id,index_user,user_id,rating
0,0,5333,509,27448,5
1,1,3206,12,6630,4
2,1,3206,163,17434,5
3,1,3206,168,23576,5
4,1,3206,178,13776,3
5,1,3206,260,41577,5
6,1,3206,293,45493,3
7,1,3206,295,3918,4
8,1,3206,443,18045,2
9,1,3206,451,14177,5


In [69]:
n_users = df_notes['user_id'].nunique()
n_livres = df_notes['book_id'].nunique()
print('Les dimensions de df_notes sont de : ' + str(df_notes.shape))
print("Nombre d'utilisateurs : " + str(n_users))
print("Nombre de livres : " + str(n_livres))

Les dimensions de df_notes sont de : (23437, 5)
Nombre d'utilisateurs : 608
Nombre de livres : 1000


## Création des sets d'entraînement et de test

Tout comme nous l'avons fait pour les thèmes préalablement (Cf. Partie I du filtrage collaboratif), on sépare le dataset en deux sets distincts : un pour l'entraînement de notre modèle de prédiction et un pour tester ce modèle. On garde 25% des données pour le set de test.

In [43]:
train_data, test_data = train_test_split(df_notes, test_size=0.25)
train_data.head()

,index_livre,book_id,index_user,user_id,rating
3603,146,125,209,22164,4
2036,95,2607,461,15194,3
21413,912,1576,155,25182,5
3567,146,125,104,30283,5
19480,830,1018,216,17804,3


## Filtrage collaboratif

On se base toujours sur les deux mêmes modèles pour le filtrage collaboratif : le "**memory-based**" et le "**model-based**".

Pour commencer, on crée une matrice utilisateur-livre pour l'entraînement du modèle de prédiction.

In [51]:
train_data_matrix = np.zeros((n_users,n_livres))

In [52]:
for line in train_data.itertuples():
    train_data_matrix[line[3], line[1]] = line[5]
train_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

On crée ensuite une matrice utilisateur-thème pour tester le modèle.

In [53]:
test_data_matrix = np.zeros((n_users,n_livres))

In [54]:
for line in test_data.itertuples():
    test_data_matrix[line[3], line[1]] = line[5]
test_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Filtrage collaboratif avec la méthode Memory-Based 

L'idée sous-jacente derrière le modèle dit "**memory-based**" est de calculer et d'utiliser les **similarités** entre utilisateurs et/ou items -ici les thèmes- et d'utiliser ces facteurs comme des "poids"  permettant la prédiction d'un thème, d'une note attribuée à un livre, ou autre. 

Nous allons tester les deux types de filtrage collaboratif:

* Item-Item 
* Utilisateur-Item 

Nous utilisons le coefficient de similarité. Pour cela, nous importons la fonction "pairwise_distances" de Scikit-Learn. 

On calcule d'abord la similarité entre les utilisateurs.

In [55]:
from sklearn.metrics import pairwise
user_similarity = pairwise.cosine_similarity(train_data_matrix)

In [56]:
user_similarity

array([[1.        , 0.27786086, 0.29377887, ..., 0.07473288, 0.13880146,
        0.11708833],
       [0.27786086, 1.        , 0.18293123, ..., 0.14522597, 0.10725293,
        0.13590262],
       [0.29377887, 0.18293123, 1.        , ..., 0.20008829, 0.26288237,
        0.16432203],
       ...,
       [0.07473288, 0.14522597, 0.20008829, ..., 1.        , 0.18764551,
        0.15313109],
       [0.13880146, 0.10725293, 0.26288237, ..., 0.18764551, 1.        ,
        0.19752469],
       [0.11708833, 0.13590262, 0.16432203, ..., 0.15313109, 0.19752469,
        1.        ]])

Calcul de la similarité entre les livres :

In [59]:
item_similarity = pairwise.cosine_similarity(train_data_matrix.T)

In [60]:
item_similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

On définit une méthode pour réaliser les prédictions. 

In [61]:
def predict(choices, similarity, kind='user'):
    
    sum_sim = np.array([np.abs(similarity).sum(axis=1)])
    sum_sim[sum_sim == 0] = 1    
    if kind == 'user':
        return similarity.dot(choices) / sum_sim.T
    elif kind == 'item':
        return choices.dot(similarity) / sum_sim

Cette méthode permet de prédire les livres susceptibles d'intéresser un utilisateur. Soit elle prend en considération les livres qui lui plaisent déjà, soit elle regarde les thèmes de prédilection d'autres utilisateurs ayant donné des réponses similaires.

In [62]:
item_prediction = predict(train_data_matrix, item_similarity, 'item')

In [63]:
item_prediction[0:5,0:3]

array([[0.09012529, 0.22497681, 0.2589533 ],
       [0.20688677, 0.20361765, 0.62090655],
       [0.12507581, 0.27639341, 0.14598705],
       [0.04970811, 0.21190072, 0.22208339],
       [0.21616734, 0.24452245, 0.10361473]])

In [64]:
user_prediction = predict(train_data_matrix, user_similarity, 'user')
user_prediction[0:5,0:3]

array([[0.0025917 , 0.0528627 , 0.04487075],
       [0.00677169, 0.03871761, 0.0808874 ],
       [0.00404404, 0.05115041, 0.02118263],
       [0.00171438, 0.04864771, 0.03221118],
       [0.0089254 , 0.05878382, 0.01725629]])

On mesure la performance du modèle avec le calcul de la RMSE (root-mean-square error), c'est-à-dire la racine carrée de l'erreur quadratique. Cette méthode compare les vraies réponses aux réponses prédites par notre modèle.

In [65]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, true_value):
    prediction = prediction.flatten()
    true_value = true_value.flatten()
    return sqrt(mean_squared_error(prediction, true_value))

RMSE pour la prédiction basée sur la comparaison entre les utilisateurs. 

In [66]:
user_CF_RMSE = rmse(user_prediction, test_data_matrix)
print('RMSE basée sur les utilisateurs : ', user_CF_RMSE)

RMSE basée sur les utilisateurs :  0.42418025225419365


In [67]:
item_CF_RMSE = rmse(item_prediction, test_data_matrix)
print('RMSE basée sur les thèmes : ', item_CF_RMSE)

RMSE basée sur les thèmes :  0.45386822381131486


## Filtrage collaboratif avec la méthode Model-based

La même logique développée dans la partie précédente (cf. I.1 memory-based collaborative filtering) peut être utilisée dans la méthode dite "model-based" : les similarités entre utilisateurs et/ou items peuvent être calculées et associées à un *modèle*, et on peut ensuite utiliser ce modèle pour faire nos prédictions. 

Le filtrage collaboratif dit "model-based" repose sur la factorisation de matrice. 

Nous allons utiliser un algorithme "SVD-based" permettant de réduire les dimensions de notre set de données et de guarder les caractéristiques principales, c'est-à-dire déterminantes de nos prédictions.

On regarde la proportion d'absence de données dans notre matrice.

In [70]:
sparsity=round(1.0-len(df_notes)/float(n_users*n_livres),3)
print('Taux de 0 dans la matrice :' +  str(sparsity*100) + '%')

Taux de 0 dans la matrice :96.1%


Decompose the train_data_theme_matrix using the SVD method.

In [71]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 13)

Crée une matrice diagonale.

In [72]:
s_diag_matrix=np.diag(s)

Compute the rating predictions from the decomposition values.

In [73]:
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

Compute the model RMSE.

In [75]:
k = np.arange(1,10) 
print(k)
for k in k:
    u, s, vt = svds(train_data_matrix, k = k)
    s_diag_matrix=np.diag(s)
    X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD-based CF RMSE (k={}): {}'.format(k, str(rmse(X_pred, test_data_matrix))))

[1 2 3 4 5 6 7 8 9]
SVD-based CF RMSE (k=1): 0.42129094728953215
SVD-based CF RMSE (k=2): 0.4294439057195919
SVD-based CF RMSE (k=3): 0.43681393777181426
SVD-based CF RMSE (k=4): 0.44363899960635694
SVD-based CF RMSE (k=5): 0.4507822883434278
SVD-based CF RMSE (k=6): 0.45809242382741466
SVD-based CF RMSE (k=7): 0.4635105890397339
SVD-based CF RMSE (k=8): 0.4690355653019209
SVD-based CF RMSE (k=9): 0.4749943069111912


### PCA avec Scikit-Learn 

In [76]:
# on importe la librairie permettant la réduction de dimension de notre set de données sur les thèmes
from sklearn.decomposition import PCA 

Ici, on détermine précisemment le nombre minimum de dimensions à garder pour préserver au moins 95% de la variance caractérisant notre set de données.

In [87]:
# On détermine d, le nombre de dimensions après PCA
pca = PCA(n_components=100, svd_solver='arpack')
pca.fit(train_data)
print(pca.explained_variance_ratio_)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95)+1
print(cumsum)

# On trace la variance cumulative en fonction du nombre de dimensions
dim=np.arange(1,6)
plt.plot(dim, cumsum)
axes = plt.gca()
plt.axhline(y=0.95,color="red")
#plt.axvline(x=13, color="black", linestyle='--')
#axes.xaxis.set_ticks(range(1001))
#plt.axvspan(12, 13, facecolor='#2ca02c', alpha=0.3)
plt.title("Variance expliquée en fonction du nombre de dimensions")
plt.xlabel("Dimensions")
plt.ylabel("Variance cumulative expliquée (%)")
plt.gcf().set_size_inches(15, 10)
plt.show()

print ('\n nombre de dimensions du set après PCA : ' + str(d))

ValueError: n_components=100 must be between 1 and min(n_samples, n_features)=5 with svd_solver='arpack'

Cela veut dire que l'on ne peut enlever qu'une seule dimension à notre set de données si on souhaite garder assez d'informations pour la prédiction. Cependant, comme nous pouvons le voir sur le graphique, réduire le set à 12 dimensions ne nous ferait pas dépasser de beaucoup la limite de variance cumulative généralement fixée à 95% de celle du set initial.